In [2]:
!pip install scikit-plot


[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Import Libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score,roc_curve,auc
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import scikitplot as skplt

Import Data

In [12]:
train_data_classification = pd.read_csv(r"C:\Users\MSI\OneDrive\1. BiG_dAtA_Master_Degree\thesis\Code\Data\Classification\Test_jPKyvmK.csv")
test_data_classification = pd.read_csv(r"C:\Users\MSI\OneDrive\1. BiG_dAtA_Master_Degree\thesis\Code\Data\Classification\Train_eP48B9k.csv")

train_data_classification.head()

,id,customer_age,job_type,marital,education,default,balance,housing_loan,personal_loan,communication_type,day_of_month,month,last_contact_duration,num_contacts_in_campaign,days_since_prev_campaign_contact,num_contacts_prev_campaign,prev_campaign_outcome
0,id_17231,55.0,retired,married,tertiary,no,7136.0,no,no,cellular,13,aug,90.0,2.0,NaN,0,unknown
1,id_34508,24.0,blue-collar,single,secondary,no,179.0,yes,no,cellular,18,may,63.0,2.0,NaN,0,unknown
2,id_44504,46.0,technician,divorced,secondary,no,143.0,no,no,cellular,8,jul,208.0,1.0,NaN,0,unknown
3,id_174,56.0,housemaid,single,unknown,no,6023.0,no,no,unknown,6,jun,34.0,1.0,NaN,0,unknown
4,id_2115,62.0,retired,married,secondary,no,2913.0,no,no,cellular,12,apr,127.0,1.0,188.0,1,success


In [14]:
train_data_classification.describe(include="all")

,id,customer_age,job_type,marital,education,default,balance,housing_loan,personal_loan,communication_type,day_of_month,month,last_contact_duration,num_contacts_in_campaign,days_since_prev_campaign_contact,num_contacts_prev_campaign,prev_campaign_outcome
count,13564,13294.000000,13564,13483,13564,13564,13383.000000,13564,13490,13564,13564.000000,13564,13442.000000,13519.000000,2441.000000,13564.000000,13564
unique,13564,NaN,12,3,4,2,NaN,2,2,3,NaN,12,NaN,NaN,NaN,NaN,4
top,id_17231,NaN,blue-collar,married,secondary,no,NaN,yes,no,cellular,NaN,may,NaN,NaN,NaN,NaN,unknown
freq,1,NaN,2916,8123,6955,13302,NaN,7430,11314,8805,NaN,4081,NaN,NaN,NaN,NaN,11126
mean,NaN,40.450429,NaN,NaN,NaN,NaN,1341.405514,NaN,NaN,NaN,15.778089,NaN,261.171031,2.730971,223.582958,0.553966,NaN
std,NaN,10.747881,NaN,NaN,NaN,NaN,2839.670990,NaN,NaN,NaN,8.321562,NaN,261.195560,2.941338,116.196526,1.762402,NaN
min,NaN,18.000000,NaN,NaN,NaN,NaN,-6849.000000,NaN,NaN,NaN,1.000000,NaN,0.000000,1.000000,1.000000,0.000000,NaN
25%,NaN,32.000000,NaN,NaN,NaN,NaN,70.500000,NaN,NaN,NaN,8.000000,NaN,102.000000,1.000000,131.000000,0.000000,NaN
50%,NaN,39.000000,NaN,NaN,NaN,NaN,447.000000,NaN,NaN,NaN,16.000000,NaN,182.000000,2.000000,194.000000,0.000000,NaN
75%,NaN,48.000000,NaN,NaN,NaN,NaN,1466.000000,NaN,NaN,NaN,21.000000,NaN,325.000000,3.000000,323.000000,0.000000,NaN
